In [ ]:
import pandas as pd 

file_name= '2025 Fall Colloquium 출결 관리_251107.xlsx'
exception_request = pd.read_excel(file_name, sheet_name='출석예외', header=1)
online_request = pd.read_excel(file_name, sheet_name='온라인')
google_form = pd.read_excel(file_name, sheet_name='Form', header=2)
MS = pd.read_excel(file_name, sheet_name='MS (Offline)', header=2)
PHD = pd.read_excel(file_name, sheet_name='PhD (Offline)', header=2)
MS_PHD = pd.read_excel(file_name, sheet_name='Online', header=2)


In [94]:
num_col = 6 
dates = ['9/12', '9/16', '9/29', '10/14', '10/28', '11/4', '11/11', '11/18', '11/25', '12/2', '12/5', '12/9']
rename_col = {
    'Unnamed: 2': 'ID',
    'Unnamed: 3': 'Name',
}
dates = dates[:num_col]
rename_col.update({f'Unnamed: {4+f}': dates[f] for f in range(num_col)}) 

# Make IDs
MS_IDS = MS['Unnamed: 2'].astype(int).tolist()
PHD_IDS = PHD['Unnamed: 2'].astype(int).tolist()
MS_NAMES = MS['Unnamed: 3'].tolist()
PHD_NAMES = PHD['Unnamed: 3'].tolist()

MS_IDS[:3], PHD_IDS[:3]

# id x dates 
ms_final = pd.DataFrame(index=MS_IDS, columns=['Name']+dates) 
phd_final = pd.DataFrame(index=PHD_IDS, columns=['Name']+dates) 
for date in dates:
    ms_final[date] = 'X'
    phd_final[date] = 'X'

ms_final['Name'] = MS_NAMES
phd_final['Name'] = PHD_NAMES

# Process MS, PhD, and MS+PhD data 
MS_data = MS[['Unnamed: 2', 'Unnamed: 3',]+[f'Unnamed: {4+f}' for f in range(num_col)]]
MS_data = MS_data.rename(columns=rename_col, inplace=False)
MS_data['ID'] = MS_data['ID'].astype(int)
MS_data

PhD_data = PHD[['Unnamed: 2', 'Unnamed: 3',]+[f'Unnamed: {4+f}' for f in range(num_col)]]
PhD_data = PhD_data.rename(columns=rename_col, inplace=False)
PhD_data['ID'] = PhD_data['ID'].astype(int)
PhD_data

MSPHD_data = MS_PHD[['Unnamed: 2', 'Unnamed: 3',]+[f'Unnamed: {4+f}' for f in range(num_col)]]
MSPHD_data = MSPHD_data.rename(columns=rename_col, inplace=False)
MSPHD_data['ID'] = MSPHD_data['ID'].astype(int)
MSPHD_data

,ID,Name,9/12,9/16,9/29,10/14,10/28,11/4
0,20205125,박광수,O,O,O,O,O,-
1,20215138,박영진,-,O,-,-,-,-
2,20215561,이동복,O,O,O,O,-,O
3,20218169,강민기,O,O,O,O,O,O
4,20218225,이준민,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...
237,20258355,이지민,O,NaN,NaN,O,NaN,NaN
238,20258369,정규단,O,O,O,NaN,O,O
239,20258385,최진혁,O,NaN,NaN,NaN,NaN,NaN
240,20258392,홍은빈,O,NaN,NaN,NaN,NaN,NaN


In [95]:
# Online Attendance 
online_attendance = pd.read_excel(file_name, sheet_name='온라인', header=1)
def is_before_or_equal(date1, date2):
    month1, day1 = map(int, date1.split('/'))
    month2, day2 = map(int, date2.split('/'))
    numeric1 = month1 * 100 + day1
    numeric2 = month2 * 100 + day2
    return numeric1 <= numeric2
    

def parse_date(date_str):
    if "all" in date_str.lower():
        return {key: 1 for key in dates}
    if "~" in date_str:
        start, end = date_str.split("~")
        condition = {}
        for date in dates:
            if is_before_or_equal(start, date) and is_before_or_equal(date, end):
                condition[date] = 1
        return condition
    else:
        return {date_str: 1}

online_exception = {}
for i in online_attendance.index:
    number = online_attendance.loc[i, '학번']
    print(number, online_attendance.loc[i, '회차'], parse_date(str(online_attendance.loc[i, '회차'])))
    online_exception[int(number)] = parse_date(str(online_attendance.loc[i, '회차']))

# Exception Requests 
print("Handling exception requests...")
zipped_exception_request = list(zip(exception_request['학번'].astype(int), exception_request['날짜']))
zipped_exception_request[:3]

# Google Form 
google_form_data = google_form[['Unnamed: 2', 'Unnamed: 3', ] + [f'Unnamed: {4+f}' for f in range(num_col)]] 
google_form_data = google_form_data.rename(columns=rename_col, inplace=False)
google_form_data['ID'] = google_form_data['ID'].astype(int)
google_form_data = google_form_data.set_index('ID', inplace=False)

# Yangjae IDs 
yangjae = pd.read_excel(file_name, sheet_name='양재명단', header=0)
yangjae_ids = yangjae['Unnamed: 2'].astype(int).tolist()

# Make Requirement Sheet 
requirement_sheet = {}
for ids in [MS_IDS, PHD_IDS]:
    for id in ids:
        for date in dates:
            key = str(id)+'_'+date
            if id in yangjae_ids:
                requirement_sheet[key] = 'OFFLINE'
            else:
                requirement_sheet[key] = 'ONLINE'
                
for online_id in online_exception.keys():
    for date in dates:
        key = str(online_id)+'_'+date
        requirement_sheet[key] = 'ONLINE'

requirement_sheet[list(requirement_sheet.keys())[0]]

20248325.0 9/12~11/18 {'9/12': 1, '9/16': 1, '9/29': 1, '10/14': 1, '10/28': 1, '11/4': 1}
20248263.0 ALL {'9/12': 1, '9/16': 1, '9/29': 1, '10/14': 1, '10/28': 1, '11/4': 1}
20255576.0 ALL {'9/12': 1, '9/16': 1, '9/29': 1, '10/14': 1, '10/28': 1, '11/4': 1}
20238203.0 ALL {'9/12': 1, '9/16': 1, '9/29': 1, '10/14': 1, '10/28': 1, '11/4': 1}
20248266.0 ALL {'9/12': 1, '9/16': 1, '9/29': 1, '10/14': 1, '10/28': 1, '11/4': 1}
20245607.0 9/29~9/29 {'9/29': 1}
20245622.0 9/29~9/29 {'9/29': 1}
20248277.0 10/13~12/31 {'10/14': 1, '10/28': 1, '11/4': 1}
20235617.0 10/28~12/31 {'10/28': 1, '11/4': 1}
20257058.0 10/13~12/31 {'10/14': 1, '10/28': 1, '11/4': 1}
Handling exception requests...


'OFFLINE'

In [96]:
# first add the attendance check list 
names=['ms_off', 'phd_off',' online']

for ids, result_df in zip([MS_IDS, PHD_IDS], [ms_final, phd_final]):
    for id in ids:
        for date in dates:
            requirement = requirement_sheet[str(id)+'_'+date]
            # Search The Attendance Sheet 
            for name, raw_dicts in zip(names, [MS_data, PhD_data, MSPHD_data]):
                current_val = result_df.loc[id, date]
                temp = raw_dicts[raw_dicts['ID'] == id]
                if requirement == 'OFFLINE' and 'online' not in name:
                    if len(temp[date].values) > 0:
                        replace_val = str(temp[date].values[0])
                    if current_val != '0' and 'O' in replace_val:
                        result_df.loc[id, date] = 'O'
                elif requirement == 'ONLINE':
                    if len(temp[date].values) > 0:
                        replace_val = str(temp[date].values[0])
                    if current_val != 'O' and 'O' in replace_val:
                        result_df.loc[id, date] = 'O'
            # Google Form is not O, set to NoForm 
            
            if requirement == 'ONLINE':
                current_val = result_df.loc[id, date]
                google_form_val = google_form_data.loc[id, date]
                if google_form_val == 'O' and current_val == 'O':
                    result_df.loc[id, date] = 'O'
                elif google_form_val == 'O' and current_val != 'O':
                    result_df.loc[id, date] = 'X = Form(O) Attend(X)'
                elif google_form_val != 'O' and current_val == 'O':
                    result_df.loc[id, date] = 'X = Form(X) Attend(O)'
            
# Handling Request: Strict MAIL Information 
for request_id, request_date in zipped_exception_request:
    for ids, result_df in zip([MS_IDS, PHD_IDS], [ms_final, phd_final]):
        for id in ids:
            if id == request_id:
                result_df.loc[id, request_date] = 'O = MAIL Exception'
# Handling Online Exception


In [97]:
ms_final.to_csv('attendance_condition_ms.csv')
ms_final

,Name,9/12,9/16,9/29,10/14,10/28,11/4,9/30,12/5,2025-11-04 00:00:00,2025-10-28 00:00:00
20233960,Simon Aytes,X,X,X,X,X,O = MAIL Exception,NaN,NaN,NaN,NaN
20234117,김유영,X,X = Form(X) Attend(O),O,O,O,X = Form(O) Attend(X),NaN,NaN,NaN,NaN
20239005,조은비,O = MAIL Exception,O,X,O,O,O,NaN,NaN,NaN,NaN
20243416,석주영,O,O,O,O,O,O,NaN,NaN,NaN,NaN
20243503,오나영,O,O,X,O = MAIL Exception,O,O,O = MAIL Exception,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20258355,이지민,X,X,O = MAIL Exception,X,X,X,NaN,NaN,O = MAIL Exception,NaN
20258369,정규단,O,O,O,X = Form(O) Attend(X),O,O,NaN,NaN,NaN,NaN
20258385,최진혁,X,O,O,O,O,O,NaN,NaN,NaN,NaN
20258392,홍은빈,X,O,O,O,O,O,NaN,NaN,NaN,NaN


In [98]:
phd_final.to_csv('attendance_condition_phd.csv')
phd_final

,Name,9/12,9/16,9/29,10/14,10/28,11/4,12/2,12/5,12/9,2025-11-04 00:00:00
20205125,박광수,O,X,X,X,X,X,NaN,NaN,NaN,NaN
20215138,박영진,X,X = Form(X) Attend(O),X = Form(X) Attend(O),X,X,X,NaN,NaN,NaN,NaN
20215561,이동복,X,X,X,X,X,X,NaN,NaN,NaN,NaN
20218169,강민기,X,X,X,X,X,X,NaN,NaN,NaN,NaN
20218225,이준민,O,O,O,O,O,O,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20257053,김경호,O,O,O,O,O,O,NaN,NaN,NaN,NaN
20257057,김유준,X = Form(X) Attend(O),O,O,O,O,O,NaN,NaN,NaN,NaN
20257058,김채현,O,X = Form(X) Attend(O),X = Form(X) Attend(O),X,X = Form(X) Attend(O),O,NaN,NaN,NaN,NaN
20257070,장수혁,O,X,X,O = MAIL Exception,O,O,NaN,NaN,NaN,NaN


In [99]:
def str_to_score(string):
    if string[0] == 'O':
        return 1
    elif string[0] == 'X':
        return 0
    elif 'MAIL' in string:
        return 1

for date in dates:
    ms_final[date] = ms_final[date].apply(lambda row: str_to_score(row))
    phd_final[date] = phd_final[date].apply(lambda row: str_to_score(row))
    
date_to_index = {date: 1+idx for idx, date in enumerate(dates)}
ms_final.rename(columns=date_to_index, inplace=True)
phd_final.rename(columns=date_to_index, inplace=True)
ms_final.to_csv('final_ms.csv')
phd_final.to_csv('final_phd.csv')

In [100]:
ms_final

,Name,1,2,3,4,5,6,9/30,12/5,2025-11-04 00:00:00,2025-10-28 00:00:00
20233960,Simon Aytes,0,0,0,0,0,1,NaN,NaN,NaN,NaN
20234117,김유영,0,0,1,1,1,0,NaN,NaN,NaN,NaN
20239005,조은비,1,1,0,1,1,1,NaN,NaN,NaN,NaN
20243416,석주영,1,1,1,1,1,1,NaN,NaN,NaN,NaN
20243503,오나영,1,1,0,1,1,1,O = MAIL Exception,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20258355,이지민,0,0,1,0,0,0,NaN,NaN,O = MAIL Exception,NaN
20258369,정규단,1,1,1,0,1,1,NaN,NaN,NaN,NaN
20258385,최진혁,0,1,1,1,1,1,NaN,NaN,NaN,NaN
20258392,홍은빈,0,1,1,1,1,1,NaN,NaN,NaN,NaN


In [101]:
phd_final

,Name,1,2,3,4,5,6,12/2,12/5,12/9,2025-11-04 00:00:00
20205125,박광수,1,0,0,0,0,0,NaN,NaN,NaN,NaN
20215138,박영진,0,0,0,0,0,0,NaN,NaN,NaN,NaN
20215561,이동복,0,0,0,0,0,0,NaN,NaN,NaN,NaN
20218169,강민기,0,0,0,0,0,0,NaN,NaN,NaN,NaN
20218225,이준민,1,1,1,1,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20257053,김경호,1,1,1,1,1,1,NaN,NaN,NaN,NaN
20257057,김유준,0,1,1,1,1,1,NaN,NaN,NaN,NaN
20257058,김채현,1,0,0,0,0,1,NaN,NaN,NaN,NaN
20257070,장수혁,1,0,0,1,1,1,NaN,NaN,NaN,NaN
